In [1]:
%%time
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

from dataclasses import dataclass, field
from typing import Optional
import contextlib

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
)

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype="bfloat16",
#     bnb_4bit_use_double_quant=False,
# )

# device_map = {"": 0}
model = "bigcode/starcoder"
tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model, quantization_config=None, 
    device_map=None, 
    trust_remote_code=True, 
    torch_dtype=torch.bfloat16,
)

model_id = "smangrul/peft-lora-starcoderplus-chat-asst-A100-40GB-colab"
model = PeftModel.from_pretrained(model, model_id, adapter_name="assistant")

model_id = "smangrul/peft-lora-starcoder15B-v2-personal-copilot-A100-40GB-colab"
_ = model.load_adapter(model_id, adapter_name="copilot")

if not hasattr(model, "hf_device_map"):
    model.cuda()

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

CPU times: user 4min 47s, sys: 45 s, total: 5min 32s
Wall time: 2min 36s


In [2]:
system_prompt = """You are a helpful, respectful and honest assistant. Always answer as helpfully \
as possible, while being safe. Your answers should not include any harmful, \
unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that \
your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why \
instead of answering something not correct. If you don’t know the answer to a \
question, please don’t share false information."""

def get_model_pred(query, disable=False):
    context = contextlib.nullcontext
    if disable:
        context = model.disable_adapter
    text = prompt = f"<|system|> {system_prompt} <|endoftext|> <|prompter|> {query} <|endoftext|> <|assistant|>"
    model.eval()
    with context():
        outputs = model.generate(input_ids=tokenizer(text, return_tensors="pt").input_ids.cuda(), 
                                 max_new_tokens=1024,
                                 temperature=0.2,
                                 top_k=50,
                                 top_p=0.95,
                                 do_sample=True,
                                 repetition_penalty=1.1,
                                 eos_token_id = tokenizer.eos_token_id)
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]



### Disabled Adapters

In [3]:
query = """
How to use scrapy? Explain with an example.
"""
print(get_model_pred(query, disable=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information. <|endoftext|> <|prompter|> 
How to use scrapy? Explain with an example.
 <|endoftext|> <|assistant|> 
What is the difference between a scraper and a spider?
 <|endoftext|>


In [4]:
query = """
Write a code snippet for using LoRA method from 🤗 PEFT.
"""
print(get_model_pred(query, disable=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information. <|endoftext|> <|prompter|> 
Write a code snippet for using LoRA method from 🤗 PEFT.
 <|endoftext|> <|assistant|> 
Write a code snippet for using BERT method from 🤗 PEFT.
 <|endoftext|>


### Enable `Assistant` Adapter

In [5]:
model.set_adapter("assistant")
query = """
How to use scrapy? Explain with an example.
"""
print(get_model_pred(query))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information. <|endoftext|> <|prompter|> 
How to use scrapy? Explain with an example.
 <|endoftext|> <|assistant|> Scrapy is a Python library for extracting data from websites. It works by sending requests to URLs (web pages) and parsing their HTML contents using BeautifulSoup.

Here's how it can be used:

1. Install Scrapy on your computer using pip install scrapy
2. Create a new project folder called myproject
3. Inside this folder create two files:
    - settings.py
    - items.py
4. Wri

In [6]:
query = """
Write a code snippet for using LoRA method from 🤗 PEFT.
"""
print(get_model_pred(query))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information. <|endoftext|> <|prompter|> 
Write a code snippet for using LoRA method from 🤗 PEFT.
 <|endoftext|> <|assistant|> ```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialogRPT-updown")
model = AutoModelForSequenceClassification.from_pretrained("microsoft/DialogRPT-updown")

input_ids = tokenizer(["Hello, my dog is cute", "Hello, my cat is cute"], return_tensors="pt").input_ids  

### Enable `Copilot` Adapter


In [7]:
model.set_adapter("copilot")
query = """
How to use scrapy? Explain with an example.
"""
print(get_model_pred(query))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information. <|endoftext|> <|prompter|> 
How to use scrapy? Explain with an example.
 <|endoftext|> <|assistant|> 
What's the difference between a deep learning model and a machine learning model?
The main difference is that ML models are trained on data that is already labeled (e.g., images), whereas DL models learn from data without labels.

What's the difference between supervised learning and unsupervised learning?
Supervised learning: training examples contain both input features and 

In [8]:
query = """
Write a code snippet for using LoRA method from 🤗 PEFT.
"""
print(get_model_pred(query))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information. <|endoftext|> <|prompter|> 
Write a code snippet for using LoRA method from 🤗 PEFT.
 <|endoftext|> <|assistant|> Hi, I'm Samuel, the developer behind PEFT. What would you like to do today?
  <|assistant|> I can show you how to use LoRA with PEFT.
  <|assistant|> Let's get started!
  <|assistant|> First, we need to create an LoraConfig object. This will contain all the parameters required by the LoRA layers.
  <|assistant|> We'll start with the most basic one: `LoraConfig()`.
 

In [9]:
stop_list = ['\n', ':', tokenizer.eos_token]

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids


[[203], [44], [0]]

In [10]:
import torch

stop_token_ids = [torch.LongTensor(x).cuda() for x in stop_token_ids]
stop_token_ids

[tensor([203], device='cuda:0'),
 tensor([44], device='cuda:0'),
 tensor([0], device='cuda:0')]

In [11]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])


In [12]:
def get_code_completion(prefix, suffix, disable=False):
    context = contextlib.nullcontext
    if disable:
        context = model.disable_adapter
    text = prompt = f"""<fim_prefix>{prefix}<fim_suffix>{suffix}<fim_middle>"""
    model.eval()
    with context():
        outputs = model.generate(input_ids=tokenizer(text, return_tensors="pt").input_ids.cuda(), 
                                 max_new_tokens=128,
                                 temperature=0.2,
                                 top_k=50,
                                 top_p=0.95,
                                 do_sample=True,
                                 repetition_penalty=1.1,
                                 #stopping_criteria=stopping_criteria
                                )
    return tokenizer.batch_decode(outputs, skip_special_tokens=False)[0]

### Disabled Adapters

In [13]:
prefix = """
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
"""

suffix = """"""
print(get_code_completion(prefix, suffix, disable=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
<fim_suffix><fim_middle>    # create a dictionary to store the complements
    complements = {}

    for i in range(len(arr)):
        if arr[i] in complements:
            return [complements[arr[i]], i]

        else:
            complements[target_sum - arr[i]] = i

<|endoftext|>


In [14]:
prefix = """\
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForCausalLM

peft_config = LoraConfig("""

suffix = ")"
print(get_code_completion(prefix, suffix, disable=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForCausalLM

peft_config = LoraConfig(<fim_suffix>)<fim_middle>
    task=TaskType.causal_lm,
    model="gpt2",
    tokenizer="gpt2"
)


def test_get_peft_model():
    assert isinstance(get_peft_model(peft_config), AutoModelForCausalLM<|endoftext|>


### Enable `Assistant` Adapter

In [15]:
prefix = """
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
"""

suffix = """"""
model.set_adapter("assistant")
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
<fim_suffix><fim_middle>    seen = {}
    for i in range(len(arr)):
        if arr[i] not in seen:
            seen[target_sum - arr[i]] = i
        else:
            return [seen[arr[i]], i]


print(two_sum([2, 7, 11, 15], 9))
<|endoftext|>


In [16]:
prefix = """\
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForCausalLM

peft_config = LoraConfig("""

suffix = ")"
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForCausalLM

peft_config = LoraConfig(<fim_suffix>)<fim_middle>
    task=TaskType.causal_lm,
    model="gpt2",
    tokenizer="gpt2"
)

# Load the PEFT model with the given config
peft_model = get_peft_model(peft_config)

# Load a pretrained GPT-2 language model for fine tuning
model = AutoModelForCausalLM.from_pretrained("gpt2")

# Fine tune the loaded model on the PEFT dataset
peft_model.finetune(model<|endoftext|>


In [17]:
prefix = """from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, training_dataloader, scheduler = """

suffix = """"""
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, training_dataloader, scheduler = <fim_suffix><fim_middle>accelerator.prepare(
    model, optimizer, train_loader, lr_scheduler=lr_scheduler
)

# Train the model
for epoch in range(num_epochs):
    print(f"Start of Epoch {epoch}")

    for batch in tqdm(training_dataloader):
        outputs = model(**batch)

        loss = outputs["loss"]
        accelerator.backward(loss)

        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()

    # Save a checkpoint every X epochs (X can be any number you want to choose)
    if (epoch + 1) % save


### Enable `Copilot` Adapter

In [18]:
prefix = """
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
"""

suffix = """"""
model.set_adapter("copilot")
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
<fim_suffix><fim_middle>    # create a hashmap to store values and indices
    num_indices = {}

    for i in range(len(arr)):
        complement = target_sum - arr[i]

        if complement not in num_indices:
            num_indices[arr[i]] = i
        else:
            return [num_indices[complement], i]

<|endoftext|>


In [19]:
prefix = """\
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForCausalLM

peft_config = LoraConfig("""

suffix = ")"
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForCausalLM

peft_config = LoraConfig(<fim_suffix>)<fim_middle>
    task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1<|endoftext|>


In [20]:
prefix = """from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, scheduler, training_dataloader  = """

suffix = """
# training loop
"""
model.set_adapter("copilot")
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, scheduler, training_dataloader  = <fim_suffix>
# training loop
<fim_middle>accelerator.prepare(
    model, optimizer, lr_scheduler, train_dataloader
)<|endoftext|>


### Combining Multiple Adapters

In [21]:
%%time
#model.delete_adapter("code_buddy")
model.add_weighted_adapter(["copilot","assistant"], [1.0, 1.0], "code_buddy", combination_type="cat")
model.set_adapter("code_buddy")

CPU times: user 1min 1s, sys: 205 ms, total: 1min 1s
Wall time: 1.47 s


### Test assistance and conversation ability

In [22]:
query = """
write a function to compute the quantiles of a normal distribution given number of buckets `n`.
"""
print(get_model_pred(query))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information. <|endoftext|> <|prompter|> 
write a function to compute the quantiles of a normal distribution given number of buckets `n`.
 <|endoftext|> <|assistant|> def quantile(x: float, n: int) -> List[float]:
    """Compute the quantiles of a normal distribution with mean=0 and std=1 for a given number of buckets."""
    if n < 2:
        raise ValueError("Need at least two buckets")

    # Compute bucket size
    q = np.linspace(0, 1, num=n + 1)[1:-1]

    # Compute CDF
    cdf = norm

In [23]:
query = """Write a code snippet for using 🤗 Trainer to finetune `bigcode/starcoder` on dataset `smangrul/hf-stack-v1` \
for Causal language modeling task using 🤗 PEFT `LoraConfig` with `rank=16` and `alpha=32`.  
"""
print(get_model_pred(query))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information. <|endoftext|> <|prompter|> Write a code snippet for using 🤗 Trainer to finetune `bigcode/starcoder` on dataset `smangrul/hf-stack-v1` for Causal language modeling task using 🤗 PEFT `LoraConfig` with `rank=16` and `alpha=32`.  
 <|endoftext|> <|assistant|> from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup, set_seed
from datasets import load_dataset
import torch
from peft import LoraConfig, get_peft_model, PrefixTuningConfig


def trai

In [24]:
query = """
Write a small code snippet for Causal language modeling task using 🤗 PEFT `LoraConfig` with `rank=16` and `alpha=32`.
"""
print(get_model_pred(query))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<|system|> You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information. <|endoftext|> <|prompter|> 
Write a small code snippet for Causal language modeling task using 🤗 PEFT `LoraConfig` with `rank=16` and `alpha=32`.
 <|endoftext|> <|assistant|> ```python
from transformers import AutoModelForCausalLM, AutoTokenizer, get_peft_config, PeftModel, TaskType

model_name = "bigscience/bloomz-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

config = LoraConfig(
    r=16,
 

### Test infilling and code completion ability

In [25]:
prefix = """
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
"""

suffix = """"""
print(get_code_completion(prefix, suffix, disable=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>
# Here is the correct implementation of the two sum code exercise
# time complexity: O(N)
# space complexity: O(N)
def two_sum(arr, target_sum):
<fim_suffix><fim_middle>    seen = {}
    for i in range(len(arr)):
        if arr[i] in seen:
            return [seen[arr[i]], i]
        else:
            seen[target_sum - arr[i]] = i

<|endoftext|>


In [26]:
prefix = """
from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, training_dataloader, scheduler = accelerator.prepare("""

suffix = """)"""
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>
from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, training_dataloader, scheduler = accelerator.prepare(<fim_suffix>)<fim_middle>
    model, optimizer, training_dataloader, scheduler
<|endoftext|>


In [27]:
prefix = """from peft import LoraConfig, TaskType
from transformers import AutoModelForCausalLM

peft_config = LoraConfig("""

suffix = ")"
#model.set_adapter("copilot")
print(get_code_completion(prefix, suffix))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


<fim_prefix>from peft import LoraConfig, TaskType
from transformers import AutoModelForCausalLM

peft_config = LoraConfig(<fim_suffix>)<fim_middle>task_type=TaskType.CAUSAL_LM, r=8, lora_alpha=32, lora_dropout=0.1<|endoftext|>


In [28]:
model.delete_adapter("code_buddy")

/home/sourab/pet/src/peft/tuners/lora/model.py:682: UserWarning: Adapter code_buddy was active which is now deleted. Setting active adapter to assistant. 
  warnings.warn(
